In [1]:
import gensim
import gensim.downloader as api



w2v = api.load('word2vec-google-news-300')

for i, word in enumerate(w2v.vocab):
    if i == 10:
        break
    print(word)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
</s>
in
for
that
is
on
##
The
with
said


# Setup data
### Combine the fake data and real data.
### Sample it and randomize.

In [2]:
import pandas as pd
from pandas import concat
import numpy as np
from sklearn.model_selection import train_test_split


fake_data = pd.read_csv("../data/Fake.csv")
real_data = pd.read_csv("../data/True.csv")

# Mark data as real or fake.
fake_data["real"] = 0
real_data["real"] = 1

df = pd.DataFrame(np.random.randn(100, 2))
msk = np.random.randn(len(df)) < 0.8

train_fake, test_fake = train_test_split(fake_data, test_size=0.25)
train_real, test_real = train_test_split(real_data, test_size=0.25)

test_frames = [test_fake, test_real]
train_frames = [train_fake, train_real]

train = concat(train_frames)
test = concat(test_frames)

train = train.sample(frac=1)
test = test.sample(frac=1)

## Replace text with vectors using google word2vec

In [3]:
import torchtext.data.utils as text_utils

tokenizer = text_utils.get_tokenizer("basic_english")
train_titles = train["title"]
train_text = train["text"]

# for title in train["title"]:
#     token_title = tokenizer(title)
for sentence in train_titles.to_list():
    print(tokenizer(sentence))


# i = 0
# for title in train["title"]:
#     if i > 10:
#         break
#     print(title)
#     i += 1

# for title in train["title"]:
#     for word in title:
#         w2v.vectors(word)

['kremlin', 'says', 'russia', 'not', 'accused', 'in', 'u', '.', 's', '.', 'case', 'against', 'ex-trump', 'aides']
['huge', 'backlash', 'against', 'hillary', 'clinton’s', 'pandering', 'to', 'hispanics', '#notmyabuela']
['black', 'student', 'with', 'hot', 'glue', 'gun', 'causes', 'elite', 'liberal', 'college', 'to', 'lock-down', 'campus']
['senate', 'confirms', 'two', 'ferc', 'commissioners', ',', 'restoring', 'its', 'quorum']
['bulgaria', 'parliament', 'passes', 'anti-corruption', 'law', 'under', 'eu', 'pressure']
['john', 'kasich', 'tells', 'gop', 'chair', 'reince', 'priebus', 'to', 'go', 'f*ck', 'himself']
['trump', 'royally', 'f*cks', 'veterans', 'by', 'considering', 'sarah', 'palin', 'for', 'secretary', 'of', 'veterans', 'affairs']
['judge', 'raises', 'doubts', 'over', 'suit', 'against', 'trump', 'on', 'foreign', 'payments']
['fbi', 'agent', 'who', '‘found', 'nothing’', 'on', 'huma', 'and', 'anthony', 'weiner’s', 'laptops', 'in', 'record', 'time', 'right', 'before', 'election', 'is'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
import torch.nn as nn
import torch.nn.functional as nn

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

SyntaxError: unexpected EOF while parsing (<ipython-input-4-dff575420b26>, line 5)